

# 개요

모델 컬럼 전체중에
정확도가 가장 높은 컬럼 조합을 찾아주는 코드

RFE



# 1. 파일 업로드 및 기본 세팅

In [ ]:

# AIDU Library Import
from aicentro.session import Session
from aicentro.framework.keras import Keras as AiduFrm
aidu_session = Session(verify=False)
aidu_framework = AiduFrm(session=aidu_session)

# 데이터 파일 경로
data_path = aidu_framework.config.data_dir

import subprocess
import sys

import warnings
# 불필요한 경고 출력을 방지합니다.
warnings.filterwarnings('ignore')


#필요한 모델 다운로드
subprocess.call([sys.executable,"-m","pip","install","xgboost"])
subprocess.call([sys.executable,"-m","pip","install","seaborn"])
subprocess.call([sys.executable,"-m","pip","install","lightgbm"])
subprocess.call([sys.executable,"-m","pip","install","catboost"])
subprocess.call([sys.executable,"-m","pip","install","probatus"])


#파일 명
#파일 명

BEFORE_FILE_NAME = 'rfe_feature'
AFTER_FILE_NAME = 'rfe_feature'


# 데이터 불러오기 및 기본 처리


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd



#전처리 후 train 파일명
train_before_file_name = data_path + "/pre_data/train_preprocessing_"+BEFORE_FILE_NAME+".csv"

#전처리 후 train 파일명
train_after_file_name = data_path + "/pre_data/train_preprocessing_"+AFTER_FILE_NAME+".csv"

#featureimportance 모델 첫번쨰 파일
before_importance_dir = data_path + "/fit/feature_importance/model_Feature_Importance_"+BEFORE_FILE_NAME+".csv"

#featureimportance 모델 두번째 파일
after_importance_dir = data_path + "/fit/feature_importance/model_Feature_Importance_"+AFTER_FILE_NAME+".csv"


#featureimportance 비교파일
compare_importance_dir = data_path + "/fit/feature_importance/model_Feature_Importance_age.csv"

train_df = pd.read_csv(train_after_file_name)



In [ ]:
#학습하기위한 train과 validation 나눔
# 일반 데이터
# x_train, x_valid, y_train, y_valid = train_test_split(train_df.drop(['label_payment_yn'],axis=1), train_df['label_payment_yn'], test_size =0.05,random_state=21)
x_train, x_valid, y_train, y_valid = train_test_split(train_df.drop(['label_payment_yn'],axis=1), train_df['label_payment_yn'],random_state=21)


### 모델 학습 후 결과 시각화

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns

my_predictions = {}

colors = ['r', 'c', 'm', 'y', 'k', 'khaki', 'teal', 'orchid', 'sandybrown',
          'greenyellow', 'dodgerblue', 'deepskyblue', 'rosybrown', 'firebrick',
          'deeppink', 'crimson', 'salmon', 'darkred', 'olivedrab', 'olive', 
          'forestgreen', 'royalblue', 'indigo', 'navy', 'mediumpurple', 'chocolate',
          'gold', 'darkorange', 'seagreen', 'turquoise', 'steelblue', 'slategray', 
          'peru', 'midnightblue', 'slateblue', 'dimgray', 'cadetblue', 'tomato'
         ]


#acc를 구해서 시각화해줌
def acc_eval(name_, pred, actual):
    global predictions
    global colors

    acc = (pred==actual).mean()
    my_predictions[name_] = acc


    y_value = sorted(my_predictions.items(), key=lambda x: x[1], reverse=False)  # 정확도 내림차순으로 sort
    
    df = pd.DataFrame(y_value, columns=['model', 'acc'])
    print(df)
    min_ = df['acc'].min() -1
    max_ = 1.2
    
    length = len(df)
    
    plt.figure(figsize=(10, length))
    ax = plt.subplot()
    ax.set_yticks(np.arange(len(df)))
    ax.set_yticklabels(df['model'], fontsize=15)
    bars = ax.barh(np.arange(len(df)), df['acc'])
    
    for i, v in enumerate(df['acc']):
        idx = np.random.choice(len(colors))
        bars[i].set_color(colors[idx])
        ax.text(v+0.1, i, str(round(v, 3)), color='k', fontsize=15, fontweight='bold')
        
    plt.title('Accuracy', fontsize=18)
    plt.xlim(min_,max_)
    
    plt.show()
    
#acc를 구해서 시각화해줌(cross_val_score 사용하는 경우)
def acc_eval_add(name_, acc):
    global predictions
    global colors

    my_predictions[name_] = acc


    y_value = sorted(my_predictions.items(), key=lambda x: x[1], reverse=False)  # 정확도 내림차순으로 sort
    
    df = pd.DataFrame(y_value, columns=['model', 'acc'])
    print(df)
    min_ = df['acc'].min() -1
    max_ = 1.2
    
    length = len(df)
    
    plt.figure(figsize=(10, length))
    ax = plt.subplot()
    ax.set_yticks(np.arange(len(df)))
    ax.set_yticklabels(df['model'], fontsize=15)
    bars = ax.barh(np.arange(len(df)), df['acc'])
    
    for i, v in enumerate(df['acc']):
        idx = np.random.choice(len(colors))
        bars[i].set_color(colors[idx])
        ax.text(v+0.1, i, str(round(v, 3)), color='k', fontsize=15, fontweight='bold')
        
    plt.title('Accuracy', fontsize=18)
    plt.xlim(min_,max_)
    
    plt.show()
    
    df.to_csv(acc_data_dir,index=False)
    

In [ ]:
from sklearn.model_selection import cross_val_score
import numpy as np
import time
from sklearn.model_selection import KFold

# catboost rfe

In [ ]:
from catboost import CatBoostClassifier

from probatus.feature_elimination import ShapRFECV
import numpy as np
import pandas as pd
import lightgbm
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV

X_train, X_test, y_train, y_test = train_test_split(train_df.drop(['label_payment_yn'],axis=1), train_df['label_payment_yn'], random_state=21)


In [ ]:
#catboost 모델로 최적의 컬럼 조합 찾아줌
clf = CatBoostClassifier( n_estimators = 300)

# Run RFECV and ShapRFECV with the same parameters
rfe = RFECV(clf, step=1, scoring='roc_auc', n_jobs=-1, verbose=0).fit(X_train, y_train)

#step 수 조정으로 서치하는 횟수 감소 가능
shap_elimination = ShapRFECV(clf=clf, step=5, scoring='roc_auc', n_jobs=-1)

In [ ]:
#fit
shap_report = shap_elimination.fit_compute(X_train, y_train)

#shap_report를 통해 결과 데이터프레임 확인
shap_report

In [ ]:
#결과 출력
shap_report_df = pd.DataFrame(shap_report)
shap_report_df.to_csv(data_path + "/fit/accuracy/shap_report_jupyter_1012.csv")